# Project Overview 

To provide safety driving guidelines based off Crash Data

Identify high risk age groups

To detect patterns in Crash Data that lead to accidents

## The area's in which I look to address are:

How can this be used in the outside world?

What features did I use?

What did the visualizations tell about my data?

What does the statistical techniques conclude about my data?

What model was best fit?

## Gathered Data 
Gathered Traffic Crashes dataset from Chicago data portal


# Importing libraries

In [1]:
import pandas as pd
import sqlite3 
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, f1_score, recall_score
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import make_scorer, accuracy_score 
from matplotlib import pyplot as plt
from matplotlib import pyplot as plt
from sklearn.model_selection import cross_val_score
import seaborn as sns 
from sklearn.model_selection import train_test_split 
from sklearn.feature_selection import RFE
from sklearn.model_selection import KFold 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.feature_selection import SelectFromModel
from sklearn import metrics 
from sklearn.utils import resample
from IPython.core.interactiveshell import InteractiveShell
from sklearn.model_selection import GridSearchCV
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_rows', 200)
from sklearn import metrics
%matplotlib inline

/Users/jdanielkim123/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [ ]:
#Loading and concating data
people = pd.read_csv('Traffic_Crashes_-_People.csv')
df = pd.read_csv('Traffic_Crashes_-_Crashes.csv')
result=pd.merge(people, df, on=['CRASH_RECORD_ID'], how = 'inner')

In [ ]:
#Initial cleaning
result.drop_duplicates(subset='CRASH_RECORD_ID', inplace=True)
A = ['REPORT_TYPE','INJURIES_TOTAL','RD_NO_y','CRASH_DATE_x','PERSON_ID','LOCATION','RD_NO_x','VEHICLE_ID','CITY','STATE','DRIVERS_LICENSE_STATE','SEAT_NO','DRIVERS_LICENSE_CLASS','HOSPITAL','EMS_AGENCY','EMS_RUN_NO','PEDPEDAL_ACTION','PEDPEDAL_VISIBILITY','PEDPEDAL_LOCATION','CELL_PHONE_USE','CRASH_DATE_EST_I','LANE_CNT','INTERSECTION_RELATED_I','NOT_RIGHT_OF_WAY_I','BAC_RESULT VALUE','HIT_AND_RUN_I','PHOTOS_TAKEN_I','STATEMENTS_TAKEN_I','DOORING_I','WORK_ZONE_I','WORK_ZONE_TYPE', 'WORKERS_PRESENT_I']     
result.drop(A,axis=1,inplace=True)
result.AGE.fillna(36,inplace=True)
result.SEX.fillna('U',inplace=True)
result['SEX'] =np.where(result['SEX']== 'U','M',result['SEX'])
result['SEX'] =np.where(result['SEX']== 'X','F',result['SEX'])
B = ['LATITUDE','LONGITUDE','MOST_SEVERE_INJURY',
 'INJURIES_FATAL',
 'INJURIES_INCAPACITATING',
 'INJURIES_NON_INCAPACITATING',
 'INJURIES_REPORTED_NOT_EVIDENT',
 'INJURIES_NO_INDICATION',
 'INJURIES_UNKNOWN', 'BEAT_OF_OCCURRENCE', 'NUM_UNITS', 'STREET_NO',
 'STREET_DIRECTION',
 'STREET_NAME', 'DATE_POLICE_NOTIFIED', 'CRASH_DATE_y']
result.drop(B,axis=1,inplace=True)
result.dropna(axis=0, inplace=True)
result['AGE']=result['AGE'].map(lambda x: int(x))
result['AGE']=result['AGE'].map(lambda x: str(x))
result['AGE']=result['AGE'].map(lambda x: x.replace('-49','49'))
result['AGE']=result['AGE'].map(lambda x: int(x))
result.drop(['CRASH_RECORD_ID'],axis=1,inplace=True)
result.drop(['ZIPCODE','ALIGNMENT'],axis=1,inplace=True)

# EDA

In [ ]:
#created variable name eda
eda =result

# Anova test
Test whether there is a statistically significant difference in the average age based on their injury classification
- H0: The average age between different injury classes are all the same
- HA: The average age between different injury classes are not the same

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
formula = 'AGE ~ C(INJURY_CLASSIFICATION)'
lm = ols(formula, eda).fit()
table = sm.stats.anova_lm(lm, typ=2)
print(table)

In [ ]:
#Plotting violin chart
violin, ax = plt.subplots(figsize=(10,8))
sns.violinplot(y=result['AGE'], x=result['INJURY_CLASSIFICATION'], data=result)
ax.set(title='Is there a significant difference in the average age based on injury classification?')
plt.xticks(rotation=45)
violin.savefig('violin.png', bbox_inches='tight');


In [ ]:
#creating a table to compare distribution of age and the injury class
no_indication = eda[eda['INJURY_CLASSIFICATION']=='NO INDICATION OF INJURY']
non_incapacitating = eda[eda['INJURY_CLASSIFICATION']=='NONINCAPACITATING INJURY']
reported = eda[eda['INJURY_CLASSIFICATION']=='REPORTED, NOT EVIDENT']
incapacitating = eda[eda['INJURY_CLASSIFICATION']=='INCAPACITATING INJURY']
fatal = eda[eda['INJURY_CLASSIFICATION']=='FATAL']
injury_bac = pd.DataFrame(columns=['NO INDICATION OF INJURY','NONINCAPACITATING INJURY','REPORTED, NOT EVIDENT','INCAPACITATING INJURY','FATAL'])
injury_bac['NO INDICATION OF INJURY']=no_indication.groupby('AGE')['AGE'].count()
injury_bac['NONINCAPACITATING INJURY']=non_incapacitating.groupby('AGE')['AGE'].count()
injury_bac['REPORTED, NOT EVIDENT']=reported.groupby('AGE')['AGE'].count()
injury_bac['INCAPACITATING INJURY']=incapacitating.groupby('AGE')['AGE'].count()
injury_bac['FATAL']=fatal.groupby('AGE')['AGE'].count()
injury_bac.fillna(0,inplace=True)
injury_bac

In [ ]:
injury_bac.reset_index(inplace=True)

In [ ]:
#distribution of age and injury that is incapacitating  
plt.scatter(injury_bac['AGE'], injury_bac['INCAPACITATING INJURY'])
plt.show();

In [ ]:
#distribution of age and injury that is fatal  
plt.scatter(injury_bac['AGE'], injury_bac['FATAL'])
plt.show();

People between their 20's to 40 are most likely to get an incapitating injury or fatal.

# Chi Squared

Is the type of injury independent of the crash type?
- H0: injury is independent 
- HA: injury is not independent

In [ ]:
#creating a table to compare distribution of crash type and the injury class
no_indication = eda[eda['INJURY_CLASSIFICATION']=='NO INDICATION OF INJURY']
non_incapacitating = eda[eda['INJURY_CLASSIFICATION']=='NONINCAPACITATING INJURY']
reported = eda[eda['INJURY_CLASSIFICATION']=='REPORTED, NOT EVIDENT']
incapacitating = eda[eda['INJURY_CLASSIFICATION']=='INCAPACITATING INJURY']
fatal = eda[eda['INJURY_CLASSIFICATION']=='FATAL']
injury_bac = pd.DataFrame(columns=['NO INDICATION OF INJURY','NONINCAPACITATING INJURY','REPORTED, NOT EVIDENT','INCAPACITATING INJURY','FATAL'])
injury_bac['NO INDICATION OF INJURY']=no_indication.groupby('FIRST_CRASH_TYPE')['FIRST_CRASH_TYPE'].count()
injury_bac['NONINCAPACITATING INJURY']=non_incapacitating.groupby('FIRST_CRASH_TYPE')['FIRST_CRASH_TYPE'].count()
injury_bac['REPORTED, NOT EVIDENT']=reported.groupby('FIRST_CRASH_TYPE')['FIRST_CRASH_TYPE'].count()
injury_bac['INCAPACITATING INJURY']=incapacitating.groupby('FIRST_CRASH_TYPE')['FIRST_CRASH_TYPE'].count()
injury_bac['FATAL']=fatal.groupby('FIRST_CRASH_TYPE')['FIRST_CRASH_TYPE'].count()
injury_bac.fillna(0,inplace=True)
injury_bac

In [ ]:
from scipy import stats
from scipy.stats import chi2_contingency
stat,p,dof,expected = stats.chi2_contingency(injury_bac)
prob=.95
critical = stats.chi2.ppf(prob,dof)
critical,stat

We reject the null. We further explored and the most leathel car accidents are ones that you run into a fixed object

## Feature engineering

Rating primary cause of the crash and creating a risk percentage

In [ ]:
result['prim']=result['PRIM_CONTRIBUTORY_CAUSE'].map({'UNABLE TO DETERMINE': 3,'FOLLOWING TOO CLOSELY': 3,'FAILING TO YIELD RIGHT-OF-WAY': 4,
'IMPROPER BACKING': 1.5,
'FAILING TO REDUCE SPEED TO AVOID CRASH':3.5 ,
'IMPROPER OVERTAKING/PASSING': 3,
'WEATHER': 3,
'NOT APPLICABLE': 3  ,
'DISREGARDING STOP SIGN': 2 ,
'DISTRACTION - FROM INSIDE VEHICLE': 2,
'PHYSICAL CONDITION OF DRIVER': 5,
'IMPROPER LANE USAGE': 3,
'VISION OBSCURED (SIGNS, TREE LIMBS, BUILDINGS, ETC.)': 2,
'EXCEEDING SAFE SPEED FOR CONDITIONS': 3.5,
'IMPROPER TURNING/NO SIGNAL': 3,
'EQUIPMENT - VEHICLE CONDITION': 2.5,
'DRIVING SKILLS/KNOWLEDGE/EXPERIENCE': 2.5,
'DRIVING ON WRONG SIDE/WRONG WAY': 4,
'DISREGARDING TRAFFIC SIGNALS': 4.5,
'EXCEEDING AUTHORIZED SPEED LIMIT': 4,
'DISREGARDING ROAD MARKINGS': 1,
'ROAD ENGINEERING/SURFACE/MARKING DEFECTS': 2.5,
'EVASIVE ACTION DUE TO ANIMAL, OBJECT, NONMOTORIST': 1,
'OPERATING VEHICLE IN ERRATIC, RECKLESS, CARELESS, NEGLIGENT OR AGGRESSIVE MANNER': 4,
'TEXTING': 2,
'DISTRACTION - FROM OUTSIDE VEHICLE': 2,
'UNDER THE INFLUENCE OF ALCOHOL/DRUGS (USE WHEN ARREST IS EFFECTED)': 4.5,
'ANIMAL': 1,
'ROAD CONSTRUCTION/MAINTENANCE': 2,
'CELL PHONE USE OTHER THAN TEXTING': 1,
'DISREGARDING OTHER TRAFFIC SIGNS': 1.5,
'HAD BEEN DRINKING (USE WHEN ARREST IS NOT MADE)': 1,
'TURNING RIGHT ON RED': 2,
'DISTRACTION - OTHER ELECTRONIC DEVICE (NAVIGATION DEVICE, DVD PLAYER, ETC.)': 1,
'PASSING STOPPED SCHOOL BUS':1 ,
'DISREGARDING YIELD SIGN': 1,
'MOTORCYCLE ADVANCING LEGALLY ON RED LIGHT': .5,
'BICYCLE ADVANCING LEGALLY ON RED LIGHT': .5,
'RELATED TO BUS STOP': 1,
'OBSTRUCTED CROSSWALKS': 1})

In [ ]:
result['aggregate_rating'] = result['prim'].map(lambda x: x*2)
result['prim_rating'] = result['aggregate_rating'].map(lambda x: x/2)
result['risk_rating']= result['aggregate_rating'].map(lambda x: x/10)
result.drop(['aggregate_rating','prim_rating'],axis=1,inplace=True)

## Reassigning categorical with numbers

## INJURY_CLASSIFICATION
- NO INDICATION OF INJURY:0
- NONINCAPACITATING INJURY:2
- REPORTED, NOT EVIDENT:1
- INCAPACITATING INJURY:3
- FATAL:4

- 0 : No injury 
- 1 : Minor injury
- 2 : Major injury



Seperating injury classification into no, minor, and major injury

In [ ]:
result['INJURY_CLASSIFICATION']=result['INJURY_CLASSIFICATION'].map({'NO INDICATION OF INJURY':0,
 'NONINCAPACITATING INJURY':1,
 'REPORTED, NOT EVIDENT':1,
 'INCAPACITATING INJURY':2,
 'FATAL':2})

## Dummy Variables

In [ ]:
dummy_columns = ['PERSON_TYPE','SEX','SAFETY_EQUIPMENT','AIRBAG_DEPLOYED','EJECTION','DRIVER_ACTION','DRIVER_VISION','PHYSICAL_CONDITION','BAC_RESULT','TRAFFIC_CONTROL_DEVICE','DEVICE_CONDITION',
 'WEATHER_CONDITION',
 'LIGHTING_CONDITION',
 'FIRST_CRASH_TYPE',
 'TRAFFICWAY_TYPE','ROADWAY_SURFACE_COND','ROAD_DEFECT',
 'CRASH_TYPE',
 'DAMAGE',
 'PRIM_CONTRIBUTORY_CAUSE',
 'SEC_CONTRIBUTORY_CAUSE']
dummy = pd.get_dummies(result, columns=['PERSON_TYPE','SEX','SAFETY_EQUIPMENT','AIRBAG_DEPLOYED','EJECTION','DRIVER_ACTION','DRIVER_VISION','PHYSICAL_CONDITION','BAC_RESULT','TRAFFIC_CONTROL_DEVICE','DEVICE_CONDITION',
 'WEATHER_CONDITION',
 'LIGHTING_CONDITION',
 'FIRST_CRASH_TYPE',
 'TRAFFICWAY_TYPE','ROADWAY_SURFACE_COND','ROAD_DEFECT',
 'CRASH_TYPE',
 'DAMAGE',
 'PRIM_CONTRIBUTORY_CAUSE',
 'SEC_CONTRIBUTORY_CAUSE'],drop_first=True)

## Feature importance

In [ ]:
X = dummy.drop(['INJURY_CLASSIFICATION'],axis=1)
y=dummy['INJURY_CLASSIFICATION']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=23 )
rfc1 = RandomForestClassifier(random_state=1, class_weight='balanced',n_jobs=-1,verbose=True)
rfc1.fit(X_train,y_train)
y_pred_test_rfc1 = rfc1.predict(X_test)

In [ ]:
def plot_feature_importances(model):
    n_features = X_train.shape[1]
    plt.figure(figsize=(8,8))
    plt.barh(range(n_features), model.feature_importances_, align='center') 
    plt.yticks(np.arange(n_features), X_train.columns.values) 
    plt.xlabel('Feature importance')
    plt.ylabel('Feature')
    

features, ax = plt.subplots(figsize=(10,8))
sns.barplot( y=X_train.columns.values[-25: ], x=np.sort(rfc1.feature_importances_)[-25:], orient='h')
ax.set(title='Feature importances to model')
features.savefig('features.png', bbox_inches='tight');


# Modeling

There is a class imbalance in our target variable

## Decision Tree: Smote

In [ ]:
X = dummy.drop(['INJURY_CLASSIFICATION'],axis=1)
y=dummy['INJURY_CLASSIFICATION']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=23 )

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
sm = SMOTE(sampling_strategy= 'not minority', random_state=23)
X_train, y_train = sm.fit_sample(X_train, y_train)
dtc = DecisionTreeClassifier(random_state=1, class_weight = 'balanced')
dtc = dtc.fit(X_train,y_train)
# y_pred_train = dtc.predict(X_train)
y_pred_test = dtc.predict(X_test)

In [ ]:
print("Macro Testing F1 Score:",metrics.f1_score(y_test, y_pred_test, 
                                            average='macro',labels = [0,1,2]))
print("Micro Testing F1 Score:",metrics.f1_score(y_test, y_pred_test, 
                                            average='micro',labels = [0,1,2]))
print("Testing Accuracy Score:",metrics.accuracy_score(y_test, y_pred_test))


## Random Forest Classifier: Smote

In [ ]:
X = dummy.drop(['INJURY_CLASSIFICATION'],axis=1)
y=dummy['INJURY_CLASSIFICATION']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=23 )
sm = SMOTE(sampling_strategy= 'all', random_state=23,n_jobs=-1)
X_train, y_train = sm.fit_sample(X_train, y_train)


In [ ]:
rfc1 = RandomForestClassifier(random_state=1, class_weight='balanced',n_jobs=-1,verbose=True)
rfc1.fit(X_train,y_train)
y_pred_test_rfc1 = rfc1.predict(X_test)

In [ ]:
print("Macro Testing F1 Score:",metrics.f1_score(y_test, y_pred_test_rfc1, 
                                            average='macro',labels = [0,1,2]))
print("Micro Testing F1 Score:",metrics.f1_score(y_test, y_pred_test_rfc1, 
                                            average='micro',labels = [0,1,2]))
print("Testing Accuracy Score:",metrics.accuracy_score(y_test, y_pred_test_rfc1))

In [ ]:
rfc1.feature_importances_
np.sort(rfc1.feature_importances_)[-15:]

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.twilight):
    import itertools
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    tick_marks = np.arange(3)
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    plt.yticks(np.arange(0, 3, step=1.2))
    plt.xticks(np.arange(0, 3, step=1.2))

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 verticalalignment="top",
                 color="black" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [ ]:
cnf_matrix = confusion_matrix(y_test, y_pred_test_rfc1,labels=[0,1,2])
#np.set_printoptions(precision=2)

confusion, ax = plt.subplots(figsize=(8,8))
plot_confusion_matrix(cnf_matrix, classes=['No injury','Minor Injury','Severe Injury'],
                      title='')
ax.set(title='Random Forest SMOTE Confusion Matrix')
confusion.savefig('confusion_smote.png', bbox_inches='tight');

# Ranking importances for primary cause of the crash


In [ ]:
plt.rcdefaults()
fig, ax = plt.subplots()
people = (['FAILING TO YIELD RIGHT-OF-WAY', 'FOLLOWING TOO CLOSELY' ,'NOT APPLICABLE'
,'IMPROPER OVERTAKING/PASSING'
,'IMPROPER BACKING'
,'FAILING TO REDUCE SPEED TO AVOID CRASH'
,'IMPROPER LANE USAGE'
,'IMPROPER TURNING/NO SIGNAL'
,'DRIVING SKILLS/KNOWLEDGE/EXPERIENCE'
,'DISREGARDING TRAFFIC SIGNALS'
,'WEATHER'
,'OPERATING VEHICLE IN ERRATIC, NEGLIGENT OR AGGRESSIVE MANNER'
,'DISREGARDING STOP SIGN'
,'DISTRACTION - FROM INSIDE VEHICLE'
,'EQUIPMENT - VEHICLE CONDITION'
,'VISION OBSCURED (SIGNS, TREE LIMBS, BUILDINGS, ETC.)'
,'PHYSICAL CONDITION OF DRIVER'
,'UNDER THE INFLUENCE OF ALCOHOL/DRUGS'
,'EXCEEDING AUTHORIZED SPEED LIMIT'
,'DRIVING ON WRONG SIDE/WRONG WAY'
,'DISTRACTION - FROM OUTSIDE VEHICLE'
,'EXCEEDING SAFE SPEED FOR CONDITIONS'
,'ROAD ENGINEERING/SURFACE/MARKING DEFECTS'
,'ROAD CONSTRUCTION/MAINTENANCE'                                                         ])
y_pos = np.arange(len(people))
performance =  [46098
                    ,                                           45262
                     ,                                                 22140
                      ,                                  19847
                       ,                                            18523
                        ,                     17335
                         ,                                         16522
                          ,                                13855,
                                                  12808,
                                                        6971,
                 6740,
     4902,
                                                               4356,
                                                    2990
,                                                      2449
,                              2369
,                                                        2335
,                   2148
,                                                      1972
,                                                       1816
,                                                  1803
,                                                   1682
,                                              1273
,                                                         1068]
ax.barh( y_pos, performance )
ax.set_yticklabels(people)
ax.set_yticks(y_pos)
ax.set_title('PRIMARY CAUSE OF ACCIDENTS')
plt.show()
features.savefig('fig.png', bbox_inches='tight');

## Age plotted against risk rating

In [ ]:
plt.figure(figsize=(20,10))
sns.barplot(y=result['risk_rating'], x = result['AGE'], orient='h',data=result)
#plt.xticks(rotation=90);


## At what age are you most risk exposed to each classification of injury?

In [ ]:
graphing=result.groupby('INJURY_CLASSIFICATION').apply(lambda x:x.sort_values(by=['INJURY_CLASSIFICATION'],
                                                                    ascending=True))

In [ ]:
graph_data=graphing[-1755: ]
graph_data['AGE']
graph_data1=graphing[0:235093]
graph_data1['AGE']
graph_data2=graphing[235093:246211]
graph_data2['AGE']

In [ ]:
line, ax = plt.subplots(figsize=(10,8))
sns.lineplot(y=graph_data['risk_rating'], x = graph_data['AGE'], data=result, color = 'orange', label='major')
sns.lineplot(y=graph_data2['risk_rating'], x = graph_data2['AGE'], data=result, color ='green', label = 'minor')
sns.lineplot(y=graph_data1['risk_rating'], x = graph_data1['AGE'], data=result, color = 'blue', label = 'none')
ax.set(title='Age and risk rating according to injury classification')
line.savefig('line.png', bbox_inches='tight');

## GridSearch

In [ ]:
# X = dummy.drop(['INJURY_CLASSIFICATION'],axis=1)
# y=dummy['INJURY_CLASSIFICATION']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=23 )
# sm = SMOTE(sampling_strategy= 'all', random_state=23,n_jobs=-1)
# X_train, y_train = sm.fit_sample(X_train, y_train)
# parameters ={'criterion': ['entropy','gini'],'max_depth':range(1,10),'max_leaf_nodes':range(3,15),'n_estimators' : [100,200,'warn']}
# rfc_grid = RandomForestClassifier(random_state=1, class_weight='balanced',n_jobs=-1,verbose=True)
# grid_tree = GridSearchCV(rfc_grid, parameters, cv=5, n_jobs =-1)
# grid_tree.fit(X_train,y_train)

# print(grid_tree.best_estimator_)
# y_pred = grid_tree.best_estimator_.predict(X_test)
# print("Testing F1 Score:",metrics.f1_score(y_test, y_pred, 
#                                             average='micro'))